## My Project

In [ ]:
import torch
print(torch.__version__)
print('cuda available with GPU:',torch.cuda.get_device_name(0))
!nvcc --version

1.10.0+cu111
cuda available with GPU: Tesla K80
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
# Please rename according to google drive directory
os.chdir('/content/gdrive/MyDrive/CS5242/MyProject')
!pwd

Mounted at /content/gdrive
/content/gdrive/MyDrive/CS5242/MyProject


In [ ]:
# Import user functions
%load_ext autoreload
%autoreload 2
from networks.mlp_net import MLP

In [ ]:
# GPU Setup
if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('cuda not available')
    device = torch.device('cpu')

cuda available with GPU: Tesla K80


In [ ]:
train_data = torch.load('data/tensorized_b_n_white_data/training_data_colour_False.pt').squeeze(dim=1)
train_label =  torch.load('data/tensorized_b_n_white_data/training_labels_colour_False.pt').squeeze(dim=1)
test_data = torch.load('data/tensorized_b_n_white_data/testing_data_colour_False.pt').squeeze(dim=1)
test_label = torch.load('data/tensorized_b_n_white_data/testing_labels_colour_False.pt').squeeze(dim=1)

print(train_data.size(),train_label.size(),test_data.size(),test_label.size())

torch.Size([3861, 28, 28]) torch.Size([3861]) torch.Size([1644, 28, 28]) torch.Size([1644])


In [ ]:
import torch.nn as nn

def get_accuracy(scores, labels):
    num_data = scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches = indicator.sum()
    return 100*num_matches.float()/num_data 

In [ ]:
def sigma(x):
    return ( 2*torch.exp(x) - torch.exp(-x) ) / ( 2*torch.exp(x) + torch.exp(-x) )

class three_layer_MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(three_layer_MLP , self).__init__()
        self.layer1 = nn.Linear(  input_size, hidden_size )
        self.layer2 = nn.Linear(  hidden_size, hidden_size )
        self.layer3 = nn.Linear(  hidden_size, output_size )
    def forward(self, x):
        x = self.layer1(x)
        x = sigma(x)
        x = self.layer2(x)
        x = sigma(x)
        score = self.layer3(x)
        return score

net = three_layer_MLP(784,25,10)
print(net)

bs = 100
criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.Adam( net.parameters() , lr=0.01 )

for epoch in range(10):
    running_loss=0
    running_acc=0
    num_batches=0
    shuffled_indices=torch.randperm(1000)
    for count in range(0,1000,bs):
        optimizer.zero_grad()
        indices = shuffled_indices[count:count+bs]
        minibatch_data = train_data[indices]
        minibatch_label = train_label[indices]
        inputs = minibatch_data.view(bs,784)
        inputs.requires_grad_()
        scores = net(inputs) 
        loss = criterion(scores, minibatch_label) 
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().item()
        acc = get_accuracy( scores.detach() , minibatch_label)
        running_acc += acc.item()
        num_batches+=1
    total_loss = running_loss/num_batches
    total_acc = running_acc/num_batches
    print("Epoch={}, Train loss={}, Train accuracy={}".format(epoch+1, total_loss, total_acc))
    
net.eval()
with torch.no_grad():
    data = test_data.view(-1,784)
    labels = test_label
    scores = net(data) 
    print("Test accuracy={}".format(get_accuracy(scores.detach(),labels)))


three_layer_MLP(
  (layer1): Linear(in_features=784, out_features=25, bias=True)
  (layer2): Linear(in_features=25, out_features=25, bias=True)
  (layer3): Linear(in_features=25, out_features=10, bias=True)
)
Epoch=1, Train loss=0.649676101282239, Train accuracy=90.0
Epoch=2, Train loss=0.014983946038410068, Train accuracy=100.0
Epoch=3, Train loss=0.002452579513192177, Train accuracy=100.0
Epoch=4, Train loss=0.0011963633238337933, Train accuracy=100.0
Epoch=5, Train loss=0.0008819109934847802, Train accuracy=100.0
Epoch=6, Train loss=0.0007647749618627131, Train accuracy=100.0
Epoch=7, Train loss=0.0007069662155117839, Train accuracy=100.0
Epoch=8, Train loss=0.0006699163932353258, Train accuracy=100.0
Epoch=9, Train loss=0.0006406454951502383, Train accuracy=100.0
Epoch=10, Train loss=0.0006145312101580203, Train accuracy=100.0
Test accuracy=58.090023040771484


In [ ]:
train_data = torch.load('data/tensorized_data/training_data.pt').squeeze(dim=1)
train_label =  torch.load('data/tensorized_data/training_labels.pt')
test_data = torch.load('data/tensorized_data/testing_data.pt')
test_label = torch.load('data/tensorized_data/testing_labels.pt')

print(train_data.size(),train_label.size(),test_data.size(),test_label.size())

torch.Size([3860, 3, 28, 28]) torch.Size([3860, 1]) torch.Size([1645, 3, 28, 28]) torch.Size([1645, 1])


In [ ]:
# Saving checkpoint
import time
MODEL_NAME = 'MLP'
DATASET_NAME = 'MNIST'
out_dir = 'out/classification/'
root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
ckpt_dir = os.path.join(root_ckpt_dir, "RUN_")
print(ckpt_dir)

model = MLP(784,10)
epoch = 100

if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
torch.save(model.state_dict(), '{}.pkl'.format(ckpt_dir + "/epoch_" + str(epoch)))
